In [ ]:
# imports

import sympy as sp
import scipy.constants as sc
from typing import Any
import logging
import sys
import pandas as pd
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import decimal

# import ignore.my_labs
from my_labs.vne import PQ, VNE, Unit
from my_labs.my_numbers import fsdp
from my_labs.calculate import calculate, converted, scaled, Substitute, negative
from my_labs.my_pandas import GetData, GetDataXY, FrameSlice, SaveData
from my_labs.my_plt import Plot, SetXY, find_trendline
from my_labs.my_physics import constants, equations, materials

In [ ]:
# logging configuration

logging.basicConfig(
    # stream=sys.stdout,
    filename="logging.txt",
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [ ]:
# other values
data_dir = pl.Path.cwd() / "data"
data_dir

In [ ]:
# physical variables

Vзап = PQ(value=50, err=1, unit="cm^3", symbol="V_зап")

# ----------------------------------------------------------------

D_h = PQ(value=[[25.3, 34.6 - 9.3], [16.3, 30.4 - 14.2]], err=0.1, unit="cm", symbol='\Delta h')

# ----------------------------------------------------------------

equation_P_1 = sp.sympify("rho * g * h")
P = PQ(
    calculate(
        equation_P_1,
        subs={
            "rho": materials["water"].dencity.converted("kg/m^3"),
            "g": constants["g0"].converted("m/s^2"),
            "h": D_h.converted("m"),
        },
    ),
    unit="Pa",
    symbol="P",
)

# ----------------------------------------------------------------

Vфв = PQ(
    calculate(
        sp.sympify("P0 * V0 / P1"),
        subs={
            "P0": constants["P0"].converted("Pa"),
            "V0": Vзап.converted("m^3"),
            "P1": P.converted('Pa')[0],
        },
    ),
    unit="m^3",
    symbol="V_фв",
)
Vвб = PQ(
    calculate(
        sp.sympify("P0 * V0 / P2 - V1"),
        subs={
            "P0": constants["P0"].converted("Pa"),
            "V0": Vзап.converted("m^3"),
            "P2": P.converted('Pa')[1],
            "V1": Vфв.converted("m^3"),
        },
    ),
    unit="m^3",
    symbol="V_вб",
)

# ----------------------------------------------------------------

Pпр = PQ(value=9.7e-5, err=0, unit="torr", symbol="P_пр")
Pуст = PQ(value=2.3e-4, err=0, unit="torr", symbol="P_уст")
Pфв = PQ(value=1.0e-4, err=0, unit="torr", symbol="P_фв")

# ----------------------------------------------------------------

d = PQ(value=0.8, err=0, unit="mm", symbol="d")

r = PQ(calculate(d.symbol / 2, {d.symbol: d}), unit=d.unit, symbol="r")

T = PQ(value=23, err=1, unit="degC", symbol="T")

mu = PQ(value=29, err=0, unit="g/mol", symbol="mu")

L = PQ(value=10.8, err=0, unit="cm", symbol="L")

# ----------------------------------------------------------------

D_h.converted('cm'), Vзап.converted('l'), Vфв.converted('cm^3'), Vвб.converted('l')

In [ ]:
# W_1

equation6 = sp.sympify(
    (
        r.symbol**3
        * 4
        * sp.sqrt(2 * sp.pi * constants['R'].symbol * T.symbol / mu.symbol)
        * Pфв.symbol
    )
    / (3 * L.symbol)
    # '(4 * r^3 * sqrt(2 * pi * R * T / mu) * P_фв) / (3 * L)'
)

equation_W_1 = equation6 / (Pуст.symbol - Pпр.symbol)

equation_W_1
W_1 = PQ(
    calculate(
        equation_W_1,
        subs={
            r.symbol: r.converted('m'),
            constants['R'].symbol: constants['R'],
            T.symbol: T.converted('K'),
            mu.symbol: mu.converted('kg/mol'),
            Pфв.symbol: Pфв.converted('Pa'),
            L.symbol: L.converted('m'),
            Pпр.symbol: Pпр.converted('Pa'),
            Pуст.symbol: Pуст.converted('Pa'),
        },
    ),
    unit="m^3/s",
    symbol="W_1",
)

equation_W_1
W_1.converted('ml/s')

In [ ]:
# spreadsheet
data_file = data_dir / "data.xlsx"
df = pd.read_excel(data_file, na_filter=True, header=None)

In [ ]:
# data rows ranges
df_data_ranges = (
    (5, 56),
    (2, 26),
    (1, 51),
    (3, 28),
)
# begin1, end1 = (4, 56)
# begin2, end2 = (2, 26)
# begin3, end3 = (0, 51)
# begin4, end4 = (3, 28)

In [ ]:
# data

_err_t = 0.05  # sec
_err_P = 1e-2  # torr * 10^-5

data = [
    GetDataXY(
        df,
        x_frame_slice=FrameSlice(columns=3 * i, rows=slice(begin, end)),
        y_frame_slice=FrameSlice(columns=3 * i + 1, rows=slice(begin, end)),
        xerr=_err_t,
        yerr=_err_P,
        xkwargs={"symbol": "t", "unit": "s"},
        ykwargs={"symbol": "P", "unit": "torr * 10^-5"},
    )
    for i, (begin, end) in enumerate(df_data_ranges)
]

# for d in data:
#     d.x.err = _err_t
#     d.y.err = _err_P

# data

In [ ]:
# tables
tables_file_path = data_dir / "tables.xlsx"

tables: list[pd.DataFrame] = []

for d in data:
    tables.append(d.get_table())

SaveData(tables, file_path=tables_file_path)

tables[0]

In [ ]:
# preplotting

devacuuming_plots_indexes = (0, 2)
revacuuming_plots_indexes = (1, 3)

P0 = PQ(value=62, unit="torr * 10^-5", symbol="P_0")

eq_P = sp.ln((sp.Symbol("P") - Pпр.symbol) / P0.symbol)

subs_P = {
    Pпр.symbol: Pпр.converted("torr * 10^-5"),
    P0.symbol: P0.converted("torr * 10^-5"),
}

plot_data_devacuuming = [data[i].move_x_to_0().as_numeric() for i in devacuuming_plots_indexes]
plot_data_revacuuming = [
    data[i].converted(yequation=eq_P, ysubs=subs_P).move_x_to_0().as_numeric()
    for i in revacuuming_plots_indexes
]

plot_data = plot_data_devacuuming + plot_data_revacuuming

# del devacuuming_plots_indexes, revacuuming_plots_indexes
del P0
del eq_P, subs_P

plot_data

In [ ]:
# plot: all

title = lambda i: ("devacuuming" if i in devacuuming_plots_indexes else "revacuuming") + " " + f"({i+1})"
savefig = lambda i: data_dir / f"plot{i + 1}"
xlabel = lambda i: "t, с"
ylabel = lambda i: "P, torr * 10^-5" if i in devacuuming_plots_indexes else 'ln((P - Pпр) / P0)'

[
    Plot(
        plot_data[i],
        title=title(i),
        # xlabel=xlabel(i),
        # ylabel=ylabel(i),
        savefig=savefig(i),
        close=False,
    )
    for i, _ in enumerate(plot_data)
]

In [ ]:
# slopes
slopes_devacuuming = [
    find_trendline(p).slope
    for p in plot_data_devacuuming
]

slopes_revacuuming = [
    find_trendline(p).slope
    for p in plot_data_revacuuming
]

slopes_devacuuming, slopes_revacuuming

In [ ]:
# W_2

a = sp.Symbol('alpha')

equation_W_2 = -a * Vвб.symbol

W_2 = PQ(
    calculate(
        equation_W_2,
        subs={
            a: slopes_revacuuming,
            Vвб.symbol: Vвб.converted("m^3").medium(),
        },
    ),
    unit="m^3/s",
    symbol="W",
)

W_2.converted("ml/s").medium().repr_condensed()

In [ ]:
# Qн

W: PQ = W_2

equation_Qн = (Pпр.symbol * W.symbol - Vвб.symbol * a) / (
    constants["R"].symbol * T.symbol
)

Qн = PQ(
    calculate(
        equation_Qн,
        subs={
            Pпр.symbol: Pпр.converted("Pa"),
            W.symbol: W.converted("m^3 / s"),
            Vвб.symbol: Vвб.converted("m^3").medium(),
            a: slopes_devacuuming,
            constants["R"].symbol: constants["R"],
            T.symbol: T.converted("K"),
        },
    ),
    symbol="Q_н",
    unit="mol/s",
)

equation_Qн
Qн.medium()